# CMIP6 Consistent Time Coordinate

**Following steps are included in this script:**

1. Load netCDF files
2. Create a consistent time coordinate
3. Save and replace netcdf files

In [1]:
# ========== Packages ==========
import xarray as xr
import pandas as pd
import numpy as np
import dask
import os

### Functions

In [2]:
def consis_time(ds_dict, ref_ds):
    """
    Creates consistent time coordinate based on a reference dataset

    Args:
        ds_dict (dict): A dictionary of xarray datasets, where each key is the name of the dataset 
                        and each value is the dataset itself.
        ref_ds (xarray): A xarray dataset as reference for the consistent time coordinate

    Returns:
        dict: A dictionary with a new time coordinate depending on the reference dataset.
    """
    time = ref_ds.time
    
    for i, (name, ds) in enumerate(ds_dict.items()):
        # Create consistent time coordinate using the first time coordinate for all following models
        if not ds['time'].equals(time):
            ds['time'] = time
            # Add comment about changes to data 
            if 'log' in ds.attrs:
                log_old = ds.attrs['log']
                ds.attrs['log'] = f'Time coordinate changed to format cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True). // {log_old}'
            else:
                ds.attrs['log'] = 'Time coordinate changed to format cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True).'
        else:
            print('Time variable is already in the requested format')
            
        # Update the dictionary with the modified dataset
        ds_dict[name] = ds
            
    return ds_dict

In [3]:
def save_file(save_file, folder, save_var=True):
    """
    Save files as netCDF.

    Args:
        savefile (dict or dataset): Dictionary of xarray datasets or dataset.
        folder (string): Name of folder data is saved in.
        save_var (boolean): If True, data is saved separately for each variable. If false, one file is saved with all variables.
        

    Returns:
        nc_out: Path were data is saved in.
    """
    
    if save_var:
        for key, ds in ds_dict.items():
            for var in ds:
                # Variable to keep
                variable_to_keep = var
                dimensions_to_keep = {'time', 'lat', 'lon'}
                coordinates_to_keep = {'time', 'lat', 'lon'}

                if any('depth' in ds[var].dims for var in ds.variables):
                    dimensions_to_keep.add('depth')
                    coordinates_to_keep.add('depth')

                # Create a new dataset with only the desired variable
                ds_var = ds[[variable_to_keep]]

                # Keep only the desired dimensions
                ds_var = ds_var.isel({dim: slice(None) for dim in dimensions_to_keep.intersection(ds_var.dims)})

                # Set the desired coordinates
                coords_to_set = set(ds_var.variables).intersection(coordinates_to_keep)
                ds_var = ds_var.set_coords(list(coords_to_set))

                savepath = f'../../data/CMIP6/{ds_var.experiment_id}/{folder}/{var}/'
                filename = f'CMIP.{ds_var.source_id}.{ds_var.experiment_id}.{var}.nc'
                nc_out = os.path.join(savepath, filename)
                os.makedirs(savepath, exist_ok=True) 
                if os.path.exists(nc_out):
                    #    inp = input(f"Delete old file {filename} (y/n):")
                    #    if inp.lower() in ["y"]:
                            os.remove(nc_out)
                            print(f"File  with path: {nc_out} removed")
                    #    else:
                    #        filename = "temp_file.nc"
                    #        nc_out = os.path.join(savepath, filename)
                    #        print(f"Filename change to {filename}")

                # Save to netcdf file
                with dask.config.set(scheduler='threads'):
                    ds_var.to_netcdf(nc_out)
                    print(f"File with path: {nc_out} saved")
       
    else:
        for key in save_file.keys():
            ds_in = save_file[key]
            filename = f'CMIP.{ds_in.source_id}.{ds_in.experiment_id}.nc'
            savepath = f'../data/CMIP6/{ds_in.experiment_id}/{folder}'
            nc_out = os.path.join(savepath, filename)
            os.makedirs(savepath, exist_ok=True) 
            if os.path.exists(nc_out):
                inp = input(f"Delete old file {filename} (y/n):")
                if inp.lower() in ["y"]:
                    os.remove(nc_out)
                    print(f"File  with path: {nc_out} removed")
                else:
                    filename = "temp_file.nc"
                    nc_out = os.path.join(savepath, filename)
                    print(f"Filename change to {filename}")

            # Save to netcdf file
            with dask.config.set(scheduler='threads'):
                ds_in.to_netcdf(nc_out)

    return nc_out

### 1. Load netCDF files

In [4]:
# ========= Define period, models and path ==============
folder="raw"
variable='ps' # ['evspsbl', 'gpp', 'huss', 'lai', 'mrro', 'mrsol', 'pr', 'tran', 'tsl', 'ps']
experiment_id = 'historical'
source_id = ['TaiESM1', 'BCC-CSM2-MR',  'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR', 'UKESM1-0-LL', 'MPI-ESM1-2-LR', 'CESM2-WACCM', 'NorESM2-MM']

#source_id = ['AWI-ESM-1-1-LR']#['AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CanESM5', 'CESM2-FV2', 'CESM2-WACCM-FV2', 'CESM2-WACCM', 'CESM2', 'CNRM-CM6-1-HR','CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR', 'NorESM2-MM','MPI-ESM1-2-LR', 'TaiESM1', 'UKESM1-0-LL'] # 'SAM0-UNICON''AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CanESM5', 'CESM2-FV2', 'CESM2-WACCM-FV2', 'CESM2-WACCM', 'CESM2', 'CNRM-CM6-1-HR','CNRM-CM6-1', 'CNRM-ESM2-1', ''IPSL-CM6A-LR', 'NorESM2-MM', 'TaiESM1', 'UKESM1-0-LL', 'SAM0-UNICON'], ['BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1-HR','NorESM2-MM', 'SAM0-UNICON', 'TaiESM1'] 
savepath = f'../../data/CMIP6/{experiment_id}/{folder}/{variable}' #change if you don't have your data preprocessed yet

# ========= Use Dask to parallelize computations ==========
dask.config.set(scheduler='processes')

# ========= Create a helper function to open the dataset ========
def open_dataset(filename):
    ds = xr.open_dataset(filename)
    return ds

# ========= Create dictionary using a dictionary comprehension and Dask =======
ds_dict, = dask.compute({model: open_dataset(os.path.join(savepath, f'CMIP.{model}.{experiment_id}.{variable}.nc'))
                        for model in source_id})

In [5]:
# ========= Have a look into the dictionary =======
print(list(ds_dict.keys()))
ds_dict[list(ds_dict.keys())[0]] # .tsl.isel(time=400, depth=0).plot()

['TaiESM1']


<xarray.Dataset>
Dimensions:  (time: 1980, lat: 192, lon: 288)
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    tran     (time, lat, lon) float32 ...
Attributes: (12/53)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 0671-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     171550.0
    ...                        ...
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/98a43e52-7cbb-40c3-85b8-5c2195f4f60d
    intake_esm_varname:        mrsol
    intake_esm_dataset_key:    CMIP.TaiESM1.historical.Emon.gn
    log:                       Dropped: member_id.

In [7]:
unique_values = ds_dict[list(ds_dict.keys())[0]][variable].to_series().unique()
print(unique_values)

[ 0.0000000e+00            nan -3.4177360e-06 ... -6.1483814e-07
 -4.0925994e-07 -2.3161016e-07]


### 2. Create consistent time coordinates

In [5]:
# =========== Create consistent time coordinate ==========

# Define reference dataset with desired time coordinate and set variables as some variables seem to have different time coordinates even from the same model
ref_ds = xr.open_dataset(f'../../data/CMIP6/{experiment_id}/raw/pr/CMIP.NorESM2-MM.{experiment_id}.pr.nc')

# Apply time coordinate on dictionary
ds_dict = consis_time(ds_dict, ref_ds)

Time variable is already in the requested format
Time variable is already in the requested format
Time variable is already in the requested format
Time variable is already in the requested format


### 3. Save and replace netcdf files

In [6]:
folder='preprocessed'

In [7]:
# =========== Store file and remove any former one ==========
nc_out = save_file(ds_dict, folder=folder)

File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.TaiESM1.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.BCC-CSM2-MR.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.CanESM5.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.CNRM-CM6-1.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.CNRM-ESM2-1.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.IPSL-CM6A-LR.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.UKESM1-0-LL.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.MPI-ESM1-2-LR.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.CESM2-WACCM.historical.ps.nc saved
File with path: ../../data/CMIP6/historical/preprocessed/ps/CMIP.NorESM2-MM.historical.ps.nc saved


In [ ]:
# =========== Check stored file ==============
xr.open_dataset(nc_out)